In [ ]:
#Intially i used PCA to distill the features into principal components that can account for most of data variances and deal with correlated features. 
#However PCA can't handle redundant features that have little values in predicitng potential customers. 
#When I completed part 3 using XGBoost Classifier, I could extract the most important features in predicting potential customers.

#I would like to find out if there is an optimal number of important features for predicting potential customers. 
#Therefore using the best hyperparameters combination, now I can retrain my model. First i used the trained model in the 
#supervised learning part to reaarange the features from most important to least important

import pickle
xgbcl = pickle.load(open("/home/workspace/xgbcl-best-model-for-mailout-train/xgbcl model for mailout train best model with loss 0.235243", "rb"))
sorted = np.argsort(xgbcl.feature_importances_)[::-1]

impt_variables = []
column_names = train_data.columns

for i in sorted:
    impt_variables.append(column_names[i])

In [ ]:
#Next I would like to find out the optimal number of most important features to include in K-Means analysis
#Therefore using the best hyperparameters combination, now I can retrain my model. I started training with 10 most important features and loop through additional ten features till I reached 270 features.
# I will train my model on AZDIAS and CUSTOMER and predict on MAILOUT TRAIN

#I will label AZDIAS as 0 and CUSTOMER dataset as 1
response = np.empty(1082873)
response[:891221]=0
response[891221:]=1
train_data = pd.concat([train_data.reset_index(drop=True),pd.DataFrame(response)], axis=1)


#Next since I will be using MAILOUT_TRAIN as validation dataset, I would have to clean the MAILOUT_TRAIN dataset. Therefore i will perform the same preprocessing steps as before. I have aggregrated all the preprocessing
#codes into a single function
def cleaning_data(train_data):
    columns_to_drop = ['ALTER_KIND1', 'ALTER_KIND2', 'ALTER_KIND3', 'ALTER_KIND4', 'EXTSEL992', 'KK_KUNDENTYP']
    train_data.drop(columns = columns_to_drop, inplace=True)
    k = 0
    for i,j in var_unknown.items():
        if i in columns_to_remove:
            continue
        if i in columns_to_rename:
            train_data[columns_renamed[k]] = train_data[columns_renamed[k]].replace(j, np.nan)
            continue
        train_data[i] = train_data[i].replace(j, np.nan)
    #'RELAT_AB' need to replace 9 with nan
    train_data['RELAT_AB'] = train_data['RELAT_AB'].replace([9], np.nan)
    columns_to_drop_1 = ['AGER_TYP', 'KBA05_BAUMAX', 'TITEL_KZ']
    train_data.drop(columns = columns_to_drop_1, inplace=True)

    #dropping 'LP_STATUS_GROB', 'LP_FAMILIE_GROB' because similar to 'LP_STATUS_FEIN' and 'LP_FAMILIE_FEIN' 
    train_data.drop(columns = ['LP_STATUS_GROB', 'LP_FAMILIE_GROB'], inplace=True)
    #dropping LNR
    train_data.drop(columns = ['LNR'], inplace=True)

    #reengineering features
    #PLZ8_BAUMAX, how many family homes
    train_data['PLZ8_BAUMAX'] = train_data['PLZ8_BAUMAX'].replace([5], 0)
    #if person lives in business building or not
    train_data['PLZ8_BAUMAX_BIZ'] = train_data['PLZ8_BAUMAX'].replace([0,1,2,3,4], [1,0,0,0,0])
    #main of avant-garde
    train_data['PRAEGENDE_MAIN_OR_AVANT'] = train_data['PRAEGENDE_JUGENDJAHRE'].replace([i for i in range(1,16)], [0,1,0,1,0,1,1,0,1,0,1,0,1,0,1])
    #40ies,50ies,60ies,70ies,80ies,90ies movements
    train_data['PRAEGENDE_JUGENDJAHRE'] = train_data['PRAEGENDE_JUGENDJAHRE'].replace([i for i in range(1,16)], [1,1,2,2,3,3,3,4,4,5,5,5,5,6,6])

    #CAMEO_INTL
    train_data['CAMEO_INTL_2015'] = train_data['CAMEO_INTL_2015'].replace(['X', 'XX'], np.nan)
    train_data['CAMEO_INTL_2015'] = train_data['CAMEO_INTL_2015'].astype(float)
    train_data['CAMEO_DEUG_2015'] = train_data['CAMEO_DEUG_2015'].replace(['X', 'XX'], np.nan)
    train_data['CAMEO_DEUG_2015']  = train_data['CAMEO_DEUG_2015'].astype(float)
    #household income
    train_data['CAMEO_HOUSEHOLD'] = train_data['CAMEO_INTL_2015'].replace([11, 12, 13, 14, 15, 21, 22, 23, 24, 25, 31, 32, 33, 34, 35,
                                                                                  41, 42, 43, 44, 45, 51, 52, 53, 54, 55],[1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5,5,5,5])
    #family type 
    train_data['CAMEO_FAMILY_TYPE'] = train_data['CAMEO_INTL_2015'].replace([11, 12, 13, 14, 15, 21, 22, 23, 24, 25, 31, 32, 33, 34, 35,
                                                                                   41, 42, 43, 44, 45, 51, 52, 53, 54, 55], [1,2,3,4,5,1,2,3,4,5,1,2,3,4,5,1,2,3,4,5,1,2,3,4,5])

    #OST_WEST_KZ encode 'W' to 0 and '0' to 1
    train_data['OST_WEST_KZ'] = train_data['OST_WEST_KZ'].replace(['W', 'O'], [0,1]) 
    #one hot encoding of 'ANREDE_KZ'
    train_data = pd.get_dummies(train_data, columns=['ANREDE_KZ'], drop_first=True)


    train_data.drop(columns = ['CAMEO_INTL_2015'], inplace=True)
    #for WOHNLAGE, replace 0 with nan
    train_data['WOHNLAGE'] = train_data['WOHNLAGE'].replace([0], np.nan)
    #create new WOHNLAGE feature, urban or rural
    train_data['WOHNLAGE_REGION'] = train_data['WOHNLAGE'].replace([1,2,3,4,5,7,8],[0,0,0,0,0,1,1])

    #dropping CAMEO_DEU, too many categories 
    train_data.drop(columns = ['CAMEO_DEU_2015'], inplace=True)
    #dropping LP_LEBENSPHASE_FEIN, too many categories
    train_data.drop(columns = ['LP_LEBENSPHASE_FEIN'], inplace=True)
    #dropping D19_LETZTER_KAUF_BRANCHE, duplicated information
    train_data.drop(columns=['D19_LETZTER_KAUF_BRANCHE'], inplace=True)

    #converting 'EINGEFUEGT_AM' to total months
    eingefuegt = np.empty(shape=(train_data.shape[0],1))
    k = 0
    train_data['EINGEFUEGT_AM'] = train_data['EINGEFUEGT_AM'].fillna(-1)
    for i in train_data['EINGEFUEGT_AM']:
        if i != -1:

            yrmthday, zeros = i.split()
            mth_1 = (2019 - (int(yrmthday[0:4]) + 1)) * 12
            mth_2 = 12 - int(yrmthday[5:6])
            mth_3 = (30 - int(yrmthday[8:10])) / 30
            mth_total = mth_1 + mth_2 + mth_3
            eingefuegt[k] = mth_total
        else:
            eingefuegt[k] = -1
        k += 1




    train_data.drop(columns =['EINGEFUEGT_AM'] , inplace=True)
    train_data['EINGEFUEGT_MODIFIED'] = pd.DataFrame(eingefuegt)

    train_data['EINGEFUEGT_MODIFIED'] = train_data['EINGEFUEGT_MODIFIED'].replace([-1],train_data['EINGEFUEGT_MODIFIED'].mode().values)
    
    return train_data

#reading in MAILOUT TRAIN
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';', index_col=0)
#perform data cleaning and preprocessing
mailout_train = cleaning_data(mailout_train)
#since the 'RESPONSE' column is not at the end column, i recreated the "RESPONSE" column and rename it 'TARGET'
mailout_train['TARGET'] = mailout_train['RESPONSE']
mailout_train.drop(columns=['RESPONSE'], inplace=True)

In [ ]:
#I will create 2 new variables representing AZDIAS + CUSTOMER response varialbe and MAILOUT TRAIN response variable
#I started training with 10 most important features and loop through additional ten features till I reached 270 features.

roc_pred = {}

for i in range(10, 280, 10):   
 
    import xgboost as xgb
    from sklearn.metrics import roc_auc_score
    # I found tnis hyperparameter combination to yield the best result in UDACITY + ARVATO: Identify Customer Segments Kaggle competition AUC: 0.88149
    #Therefore it is used here. XGBoost algorithm is used here since it gives the best result in Kaggle
    xgbcl1 = xgb.XGBClassifier(colsample_bytree = 0.776792351030805, gamma = 1.0386435777290857, learning_rate = 0.10327932030577727, max_depth = 17, min_child_weight = 5, n_estimators = 571, reg_alpha = 0.19625707362167116, scale_pos_weight = 48, subsample = 0.8894945842180019, tree_method = 'gpu_hist', sampling_method = 'gradient_based')
    
    #the validation dataset
    eval_set = [(mailout_train.iloc[:,:i], y_test)]
    #fitting XGBClassifier to AZDIAS + CUSTOMER dataset
    xgbcl1.fit(train_data.iloc[:,:i], y_train, early_stopping_rounds=20, eval_metric="auc", eval_set=eval_set)
    #predicting on MAILOUT TRAIN
    predictions=(xgbcl1.predict_proba(mailout_train.iloc[:,:i]))[:,1]
    #recording ROC_AUC score for prediction in dictionary 
    roc_pred[i]=roc_auc_score(y_test, predictions)

In [ ]:
#After training the models, following is the list of AUC scores for the predictions on MAILOUT TRAIN using the trained models
AUC_score = [0.7817697747196178, 0.8202459291641785, 0.8445464799165011, 0.8429744745436535, 0.8476595684355834, 0.8552984216374073, 0.8518630641534309, 0.8541272533797373, 
0.8531446531128671, 0.8567158158771899, 0.856455413516114, 0.8566308019045966, 0.8566638505880538, 0.8554695128110165, 0.8583080447406519, 0.8610451934101102, 
0.8575297393849932, 0.861461403036226, 0.8597985359344625, 0.8598775027954048, 0.8578174977273492, 0.8595900767119307, 0.8565992373108118, 0.8549426831278054,
0.7928243156796069, 0.7855457861599557, 0.7781992100212822]

#plotting the AUC scores versus number of most important features used in model training.
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(13, 6))

feature = [i for i in range (10,280,10)]

plt.bar(feature, AUC_score)

plt.ylabel('ROC_AUC')
plt.xlabel('Number of most important features')
plt.ylim([0.75, 0.863])
plt.xticks(feature)
plt.show()

#plot shows that selecting the 60 most important features is optimal, therefore we will be using the 60 most important features in performing K-Means analysis
# it seems that using only 30 mostcimportant features can produce a sufficiently robust predictive model.
train_data = train_data[impt_variables][0:60]

In [ ]:
#First, we separate the train data back into AZDIAS and CUSTOMER

customer_pop = train_data.iloc[891221:,:]
train_data = train_data.iloc[:891221,:]

#finding number of rows with minimum number of missing values
missing3 = []
for i in range(0,30,1):
    l = 0    
    n = 0
    
    for j in (missing1):
        if j >= i: 
            l += 1
        
    missing3[i] = l

#plotting bar graph of number of row with minimum no of missing rows from 1 until 14
fig = plt.figure(figsize=(13, 6))
feature1 = [i for i in range (1,14,1)]
quantity1 = [ 329826, 321895, 320564, 318123, 157664, 155907, 125949, 124394, 124203, 117147, 107018, 106747, 106575]
plt.bar(feature1 ,quantity1)

plt.ylabel('Number of rows with missing values')
plt.xlabel('No of features with missing value')
plt.xticks(feature1)
plt.show()

#I decided to remove rows with minimun no of missing row = 5 since i only retained the 60 most important features
train_data1 = train_data.dropna(axis=0,thresh=train_data.shape[1]-5)
customer_pop1 = customer_pop.dropna(axis=0,thresh=train_data.shape[1]-5)

#However for completeness of analysis, I thought that maybe I should perform K-Means analysis on dataset with 30 and 45 most important features
#To ensure similar rows with missing values as the dataset with 60 features are removed, i find the rows index that are to be removed

index_to_remove = set(train_data.index.to_list()) - set(train_data1.index.to_list())
index_to_remove_1 = set(customer_pop.index.to_list()) - set(customer_pop1.index.to_list())

#for 30 most important features
customer_pop_30 = customer_pop.iloc[:,:30].drop(list(index_to_remove_1))
train_data_30 = train_data.iloc[:,:30].drop(list(index_to_remove))
#for 45 most important features
customer_pop_45 = customer_pop.iloc[:,:45].drop(list(index_to_remove_1))
train_data_45 = train_data.iloc[:,:45].drop(list(index_to_remove))

In [ ]:
#before that PCA analysis is still useful for identifying interesting features, so we will conduct PCA here
def pca_fit(df):
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import normalize
    #normalize data
    df = normalize(df)
    #fit PCA on data
    pca = PCA().fit(df)
    
    #only select components that cumulatively exceed 0.9 explaiend variance ratio
    k = 0
    m = 1
    for i in pca.explained_variance_ratio_:
        k += i
        if k > 0.9:
            break
        m += 1

    no_comp = m
    
    #fit PCA again on data with the desired number of component
    df = PCA(n_components=m).fit_transform(df)
    return pca, df

pca, train_data_transformed = pca_fit(train_data)

#Next we will find interesting variables identified by PCA

# the set should contain the variable's string name (as found in X.colummns)
def find_interesting_variables(pca, length_pca_component, top_length, feature_names):
    interesting_variables = set()
 
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import normalize
    
    #how many PCA component to analyse
    component = pca.components_[0:length_pca_component]
    
    #how many interesting variables sorted by most interesting to least interesting to return
    for i in range(len(component)):
    
        feature_with_magnitude = dict(zip(feature_names, component))
        sorted_feature_with_magnitude = sorted(feature_with_magnitude.items(), key=lambda item: item[1])
        j = 0
        for k,l in  sorted_feature_with_magnitude.items():
            if j == top_length:
                break
            interesting_variables.add(k)
            j += 1
                    
    return interesting_variables

interesting_features = find_interesting_variables(pca, length_pca_component = 5, top_length = 8, feature_names = train_data.columns)

In [ ]:
#Now we will perform K-Means analysis on 30,45 and 60 most important features dataset. I choose not to use elbow method to determine optimal number of clusters
#as the objective of customer segmentation differs from elbow merhod objective.
#instead i will conduct trial and errors and conduct K-Means analysis from 4 clusters to 15 clusters. Optimal number of cluster is chosen based on when a particular cluster
#contain a significant proportion of examples in CUSTOMER dataset.

from sklearn.cluster import KMeans
for i in range(4,16):
    km = KMeans(n_clusters=i , n_init=i+3, verbose=3, max_iter=500)
    predicted_cluster_azdias = km.fit(train_data)
    predicted_cluster_customer = km.predict(customer_pop)
    
    segment_count = {}
    for i in range(count):
        segment_count[i] = np.count_nonzero(array==i)
        
    print segment_count
#After analysing the printed segment count, I decided that the optimal number of clusters is 9

#Now i will perform K-Means analysis for 9 clusters on 30,45 and 60 most important features dataset

#function to fit K-Means algortihm on dataset and then predict the clusters that dataset belong to
def kmeans_segment(df, df2, no_cluster):
    from sklearn.cluster import KMeans
    km = KMeans(n_clusters=no_cluster , n_init=no_cluster+3, verbose=3, max_iter=500)
    km.fit(df)
    predicted_cluster_azdias = km.predict(df)
    predicted_cluster_customer = km.predict(df2)
    
    return predicted_cluster_azdias, predicted_cluster_customer


predicted_cluster_azdias, predicted_cluster_customer = kmeans_segment(train_data, customer_pop, 9)
predicted_cluster_azdias1, predicted_cluster_custome1 = kmeans_segment(train_dat_30, customer_pop_30, 9)
predicted_cluster_azdias2, predicted_cluster_customer2 = kmeans_segment(train_data_45, customer_pop_45, 9)

#function to count how many examples are contained in each cluster
def segmentation_count(array, count):
    segment_count = {}
    for i in range(count):
        segment_count[i] = np.count_nonzero(array==i)
    return segment_count

segment_count_azdias = segmentation_count(array=predicted_cluster_azdias, count=9)
segment_count_customer = segmentation_count(array=predicted_cluster_customer, count=9)

print(segment_count_azdias)
print(segment_count_customer)

segment_count_azdias1 = segmentation_count(array=predicted_cluster_azdias1, count=9)
segment_count_customer1 = segmentation_count(array=predicted_cluster_customer1, count=9)

print(segment_count_azdias1)
print(segment_count_customer1)

segment_count_azdias2 = segmentation_count(array=predicted_cluster_azdias2, count=9)
segment_count_customer2 = segmentation_count(array=predicted_cluster_customer2, count=9)

print(segment_count_azdias2)
print(segment_count_customer2)

#For 60-features dataset, cluster 0 has the most customers, for 45-features dataset, will be cluster 8 and for 30 features dataset will be cluster 2

In [ ]:
#as can be seen cluster 0 has the most customers, 54491 or 38.8% out of all customers. There are 70103 potential customers in AZDIAS in cluster 0, or 8.9% out of total in AZDIAS
#Therefore the mail order company can choose to target these potential customers.
azdias_segment = [70103, 33633, 97520, 171205, 100766, 80193, 155008, 21552, 54530]
customer_segment = [54491, 14133, 8278, 3175, 14689, 15772, 3207, 4414, 22195]

azdias_segment_prop = [0.08935896291952938, 0.042871346445552, 0.12430689220022689, 0.21823176250143403, 0.12844450676218278, 0.1022204943212961, 0.1975857541650202, 0.02747192515073103, 0.06950835553402761]
customer_segment_prop = [0.3882397366658592, 0.10069538452769426, 0.05897943770751101, 0.022621371674480244, 0.1046567963862804, 0.11237299970075666, 0.022849366601593115, 0.03144905025863175, 0.1581358564771934]

#Plot bar graph for number of examples in each cluster (total 9 clusters) for 60 most important features
fig = plt.figure(figsize=(13, 6))
X = [i for i in range(9)]
Y = [70103, 33633, 97520, 171205, 100766, 80193, 155008, 21552, 54530]
Z = [54491, 14133, 8278, 3175, 14689, 15772, 3207, 4414, 22195]
_X = np.arange(len(X))
plt.title('Number of examples in each cluster (total 9 clusters)')
plt.ylabel('Number of people')
plt.xlabel('Cluster No.')
plt.bar(_X - 0.2, Y, 0.4)
plt.bar(_X + 0.2, Z, 0.4)
plt.xticks(_X, X) # set labels manually
plt.show()


#Plot bar graph for proportion of examples in each cluster (total 9 clusters) for 60 most important features
fig = plt.figure(figsize=(13, 6))
X = [i for i in range(9)]
Y = [70103/784510, 33633/784510, 97520/784510, 171205/784510, 100766/784510, 80193/784510, 155008/784510, 21552/784510, 54530/784510]
Z = [54491/140354, 14133/140354, 8278/140354, 3175/140354, 14689/140354, 15772/140354, 3207/140354, 4414/140354, 22195/140354]
_X = np.arange(len(X))
plt.title('Proportion of examples in each cluster (total 9 clusters)')
plt.ylabel('Proportion of people')
plt.xlabel('Cluster No.')
plt.bar(_X - 0.2, Y, 0.4)
plt.bar(_X + 0.2, Z, 0.4)
plt.xticks(_X, X) # set labels manually
plt.show()

In [ ]:
#since i conducted K-Means analysis on 30,45 and 60 features dataset, I would like to find out the intersection of potential customers in AZDIAS

#First i will isolate the potential customers in AZDIAS and also customer in CUSTOMER belonging to the most overrepresented cluster

#for 30 features
predicted_cluster_azdias = pd.DataFrame(predicted_cluster_azdias)
predicted_cluster_customer = pd.DataFrame(predicted_cluster_customer)
predicted_cluster_azdias_label2 = predicted_cluster_azdias[predicted_cluster_azdias[0]==2]
predicted_cluster_customer_label2 = predicted_cluster_customer[predicted_cluster_customer[0]==2]

#for 60 features
predicted_cluster_azdias1 = pd.DataFrame(predicted_cluster_azdias1)
predicted_cluster_customer1 = pd.DataFrame(predicted_cluster_customer1)
predicted_cluster_azdias1_label0 = predicted_cluster_azdias1[predicted_cluster_azdias1[0]==0]
predicted_cluster_customer1_label0 = predicted_cluster_customer1[predicted_cluster_customer1[0]==0]

#for 45 features
predicted_cluster_azdias2 = pd.DataFrame(predicted_cluster_azdias2)
predicted_cluster_customer2 = pd.DataFrame(predicted_cluster_customer2)
predicted_cluster_azdias2 = predicted_cluster_azdias2[predicted_cluster_azdias2[0]==8]
predicted_cluster_customer2 = predicted_cluster_customer2[predicted_cluster_customer2[0]==8]

#finding intersection of potential customers between 30 ,45 and 60 most important features dataset
predicted_cluster_azdias_intersect = np.intersect1d((np.intersect1d((predicted_cluster_azdias.index), (predicted_cluster_azdias1.index))), (predicted_cluster_azdias2.index))
print(len(predicted_cluster_azdias_intersect))
print(len(predicted_cluster_azdias.index))
print(len(predicted_cluster_azdias1.index))
print(len(predicted_cluster_azdias2.index))
#there are 33720 potential customers common to 30, 45 and 60 features dataset. So, company can choose to target these potential core customers more vigorously

#finding intersection of potential customers between 45 and 60 most important features dataset.
predicted_cluster_azdias_intersect_1 = np.intersect1d((predicted_cluster_azdias1.index), (predicted_cluster_azdias2.index))
print(len(predicted_cluster_azdias_intersect_1))
print(len(predicted_cluster_azdias1.index))
print(len(predicted_cluster_azdias2.index))
#there are 53432 potential customers common to 45 and 60 features dataset. So, for a more conservative approach, company can choose to target these potential core customers more vigorously


In [ ]:
#Next, I would like to find the feature means for potential customers and customers belonging to cluster 0. Cluster 3 and 6 which represent non-customers in AZDIAS will also be examined 

train_data = pd.concat([train_data, predicted_cluster_azdias1.set_index(train_data.index)],axis=1)
customer_pop = pd.concat([customer_pop, predicted_cluster_customer1.set_index(customer_pop.index)],axis=1)

#finding feature means for cluster 0
azdias_label_0 = train_data[train_data[0]==0]
customer_label_0 = customer_pop[customer_pop[0]==0]
azdias_label_0_mean = np.mean(general_pop_label_0)
customer_label_0_mean = np.mean(customer_pop_label_0)

#finding feature means for cluster 3
azdias_label_3 = train_data[train_data[0]==3]
customer_label_3 = customer_pop[customer_pop[0]==3]
azdias_label_3_mean = np.mean(general_pop_label_3)
customer_label_3_mean = np.mean(customer_pop_label_3)

#finding feature means for cluster 6
azdias_label_6 = train_data[train_data[0]==6]
customer_label_6 = customer_po2[customer_pop[0]==6]
azdias_label_6_mean = np.mean(general_pop_label_6)
customer_6_mean = np.mean(customer_pop_label_6)

label_0_3_6_mean = pd.concat([pd.DataFrame(azdias_label_0_mean),pd.DataFrame(customer_label_0_mean),pd.DataFrame(azdias_label_3_mean),pd.DataFrame(customer_label_3_mean), pd.DataFrame(azdias_label_6_mean),pd.DataFrame(customer_label_6_mean)], axis=1)


In [ ]:
#Since this section is written after i have trained the model in supervised learning part for which i scored top in the leaderboard for the Kaggle competition with AUC 0.88149
# I am curious to find out if potential customers and non-customers predicted by my best model for the MAILOUT TRAIN dataset share common traits as those in the AZDIAS dataset

#I have already trained the model using 60 most important features that I will use to predict on MAILOUT TRAIN
xgbcl1 = pickle.load(open(pd.read_csv("/home/workspace/xgbcl model  for 60 variables identifying feature importance.pkl", "rb"))

#concatenate predictions on Mailout Train (probability and binary response) with the correct response (target variable)
predictions_1 =  xgbcl1.predict_proba(mailout_train.iloc[:,:60])[:,1]
predictions_1a = xgbcl1.predict(mailout_train.iloc[:,:60])
predictions_2 = pd.concat([pd.DataFrame(predictions_1),pd.DataFrame(predictions_1a).rename(columns={0:'1'}),pd.DataFrame(y_test)], axis=1 )
#then concatenate the above dataframe with MAILOUT TRAIN dataset of 60 most important features
mailout_train_modified = pd.concat([mailout_train.iloc[:,:60],predictions_2], axis=1)

#I have chosen a cutoff of >0.99 probability predicted on MAILOUT TRAIN to represent my pool of potential customers. 0.99 is chosen because in part 3, I found that a cutoff of 0.99
#enables me to predict most of the people who responded positively to the mailout campaign while substantially reducing the number of people wrongly predicted to respond positively
#therefore i will elaborate more on this in part 3

# this amounts to 6586 people out of which 365 are correctly predicted to be customers. (365/532 = 68.6%)
mailout_modified_99 = mailout_train_modified[mailout_train_modified[0]>0.99]
mailout_modified_99_mean = np.mean(mailout_modified_99, axis=0)

#cutoff of 0.04 is chosen on the basis that the number of non-customers predicted by my model will be similar to that of those predicted to be potential customers.
#this amount to 6898 people, of which 7 is wrongly predicted to be customers. Therefore, the number of potential customers and non-customers are pretty balanced.
mailout_modified_04 = mailout_train_modified[mailout_train_modified[0]<=0.04]
mailout_modified_04_mean = np.mean(mailout_mod_04)